In [1]:
import awkward as ak
import json
import uproot
import numpy as np
import matplotlib.pyplot as plt
import sys
import pickle
import math

from scipy.optimize import dual_annealing
from scipy import integrate
from scipy.optimize import curve_fit

import emcee
import iminuit
from iminuit.cost import UnbinnedNLL, BinnedNLL, NormalConstraint
from iminuit import Minuit

In [2]:
sys.path.append("/Users/jsen/work/Protodune/analysis/pi0_fitter")
from pi0fit.pi0_fit import Pi0Fitter
from pi0fit.shower_model import AnalyticShowerModel, BinnedShowerModel
from pi0fit.pi0_transform import Pi0Transformations
from pi0fit.pi0_fit import Pi0Fitter
from pi0fit.pi0_model import Pi0Model
import pi0fit.fitter_utilities as futil

In [3]:
with open("../config.json", 'r') as f:
        config = json.load(f)

config["analytic_shower_model"]["normalization_splines"] = "/Users/jsen/work/Protodune/analysis/pi0_fitter/etc/topo_norm_interpolation_v2.pkl"
config["binned_shower_model"]["kde_file"] = "/Users/jsen/work/Protodune/analysis/pi0_fitter/etc/charge_pdf_3d_TR_kde_bw0_20.pkl"

In [4]:
ana_model = AnalyticShowerModel(config)
bin_model = BinnedShowerModel(config)
pi0_transform = Pi0Transformations(config)
pi0_fit = Pi0Fitter(config)
pi0_model = Pi0Model(config)

Loaded KDE file for BinnedShowerModel file: /Users/jsen/work/Protodune/analysis/pi0_fitter/etc/charge_pdf_3d_TR_kde_bw0_20.pkl
3 D Ranges [(0.0, 6.8, 0.0, 0.45), (6.8, 20.0, 0.0, 0.45), (0.0, 20.0, 0.45, 10.0)]
Loaded KDE file for BinnedShowerModel file: /Users/jsen/work/Protodune/analysis/pi0_fitter/etc/charge_pdf_3d_TR_kde_bw0_20.pkl
3 D Ranges [(0.0, 6.8, 0.0, 0.45), (6.8, 20.0, 0.0, 0.45), (0.0, 20.0, 0.45, 10.0)]
Loaded KDE file for BinnedShowerModel file: /Users/jsen/work/Protodune/analysis/pi0_fitter/etc/charge_pdf_3d_TR_kde_bw0_20.pkl
3 D Ranges [(0.0, 6.8, 0.0, 0.45), (6.8, 20.0, 0.0, 0.45), (0.0, 20.0, 0.45, 10.0)]
Loaded KDE file for BinnedShowerModel file: /Users/jsen/work/Protodune/analysis/pi0_fitter/etc/charge_pdf_3d_TR_kde_bw0_20.pkl
3 D Ranges [(0.0, 6.8, 0.0, 0.45), (6.8, 20.0, 0.0, 0.45), (0.0, 20.0, 0.45, 10.0)]


In [5]:
def two_gamma_with_2conv(evt_record):
    return np.count_nonzero(evt_record["pi0_gamma_endprocess_initial"] == "conv", axis=1) == 2

def two_gamma_with_1conv(evt_record):
    return np.count_nonzero(evt_record["pi0_gamma_endprocess_initial"] == "conv", axis=1) == 1

def add_true_conv_dist(evt_record):
    conv_mask = evt_record["pi0_gamma_electron_process_initial"] == "conv" 
    min_dist_x = ak.min(abs(evt_record["pi0_startx_initial"] - evt_record["pi0_gamma_electron_startx_initial"][conv_mask]), axis=2)
    min_dist_y = ak.min(abs(evt_record["pi0_starty_initial"] - evt_record["pi0_gamma_electron_starty_initial"][conv_mask]), axis=2)
    min_dist_z = ak.min(abs(evt_record["pi0_startz_initial"] - evt_record["pi0_gamma_electron_startz_initial"][conv_mask]), axis=2)
    
    return np.sqrt(min_dist_x**2 + min_dist_y**2 + min_dist_z**2)

In [6]:
file_name = "/Users/jsen/tmp/tmp_pi0_shower/sample/single_pi0/single_pi0_updated_tree_uniform_0_2gev_n4550.root:trkUtil/points"

# Space Points
branches = ["reco_all_spacePts_X", "reco_all_spacePts_Y", "reco_all_spacePts_Z", "reco_all_spacePts_Integral"] 

# Pi0 
branches += ["pi0_pdg_initial", "pi0_trackid_initial", "pi0_startpx_initial", "pi0_startpy_initial", "pi0_startpz_initial",
            "pi0_startx_initial", "pi0_starty_initial", "pi0_startz_initial", "pi0_starte_initial", "pi0_gamma_open_angle_initial"]

# Pi0 Decay Gammas
branches += ["pi0_gamma_startx_initial", "pi0_gamma_starty_initial", "pi0_gamma_startz_initial", "pi0_gamma_endx_initial", "pi0_gamma_endy_initial",
             "pi0_gamma_endz_initial", "pi0_gamma_startpx_initial", "pi0_gamma_startpy_initial", "pi0_gamma_startpz_initial", "pi0_gamma_starte_initial",
             "pi0_gamma_pdg_initial", "pi0_gamma_motherid_initial", "pi0_gamma_trackid_initial", "pi0_gamma_process_initial", "pi0_gamma_endprocess_initial"]

# Pi0 Decay Gamma Pair e-e+ or Compton electron
branches += ["pi0_gamma_electron_startx_initial", "pi0_gamma_electron_starty_initial", "pi0_gamma_electron_startz_initial",
             "pi0_gamma_electron_startpx_initial", "pi0_gamma_electron_startpy_initial", "pi0_gamma_electron_startpz_initial",
             "pi0_gamma_electron_starte_initial", "pi0_gamma_electron_pdg_initial", "pi0_gamma_electron_motherid_initial",
             "pi0_gamma_electron_trackid_initial", "pi0_gamma_electron_process_initial"]

raw_event_record = uproot.concatenate(files={file_name}, expressions=branches)

# Only consider decays with 2 gammas
two_gamma_decay_mask = np.count_nonzero(raw_event_record["pi0_gamma_process_initial"] == "Decay", axis=1) == 2
all_event_record = raw_event_record[two_gamma_decay_mask]

tmp_zeros = ak.zeros_like(all_event_record["reco_all_spacePts_X"])
all_event_record["reco_daughter_PFP_shower_spacePts_R"] = tmp_zeros
all_event_record["reco_daughter_PFP_shower_spacePts_Theta"] = tmp_zeros
all_event_record["reco_daughter_PFP_shower_spacePts_Phi"] = tmp_zeros


"""
Add additional columns
"""
all_event_record["two_gamma_with_2conv"] = two_gamma_with_2conv(evt_record=all_event_record)
all_event_record["two_gamma_with_1conv"] = two_gamma_with_1conv(evt_record=all_event_record)
# all_event_record["true_decay_gamma_conv_startx"], all_event_record["true_decay_gamma_conv_starty"],all_event_record["true_decay_gamma_conv_startz"] = add_true_conv_dist(evt_record=all_event_record)
all_event_record["true_decay_gamma_conv_dist"] = add_true_conv_dist(evt_record=all_event_record)
all_event_record["flipped_pi0_gamma_truth"] = np.zeros_like(all_event_record["pi0_pdg_initial"], dtype=bool)

print("nEvts:", len(all_event_record))

nEvts: 4448


In [7]:
pi0_transform.set_beam_endpoint(all_event_record=all_event_record)
pi0_transform.transform_point_to_spherical(event_record=all_event_record)

In [12]:
acceptable_event_dict = {}
for event in range(0,2000):
    if len(all_event_record["pi0_gamma_starte_initial", event]) > 2: continue
    e1,e2 = all_event_record["pi0_gamma_starte_initial", event]
    epi0 = e1 + e2
    if epi0 >= 1600 and epi0 <= 2400 and e1 >= 800 and e1 <= 1200 and e2 >= 800 and e2 <= 1200:
            #print("Event:", event, "Eg:", all_event_record["pi0_gamma_starte_initial", event])
            acceptable_event_dict[event] = all_event_record["pi0_gamma_starte_initial", event]


print("nEvts", len(acceptable_event_dict))
print(acceptable_event_dict)

nEvts 53
{34: <Array [850, 898] type='2 * float64'>, 54: <Array [899, 850] type='2 * float64'>, 81: <Array [969, 909] type='2 * float64'>, 124: <Array [837, 1.02e+03] type='2 * float64'>, 146: <Array [815, 1.12e+03] type='2 * float64'>, 185: <Array [827, 1.04e+03] type='2 * float64'>, 191: <Array [838, 1.04e+03] type='2 * float64'>, 216: <Array [802, 881] type='2 * float64'>, 232: <Array [989, 908] type='2 * float64'>, 240: <Array [832, 819] type='2 * float64'>, 249: <Array [893, 872] type='2 * float64'>, 266: <Array [1.09e+03, 894] type='2 * float64'>, 280: <Array [919, 947] type='2 * float64'>, 290: <Array [1.04e+03, 803] type='2 * float64'>, 300: <Array [812, 877] type='2 * float64'>, 311: <Array [829, 801] type='2 * float64'>, 344: <Array [807, 1.14e+03] type='2 * float64'>, 365: <Array [1.06e+03, 805] type='2 * float64'>, 366: <Array [857, 833] type='2 * float64'>, 379: <Array [929, 1.04e+03] type='2 * float64'>, 471: <Array [866, 1.1e+03] type='2 * float64'>, 548: <Array [922, 83

In [9]:
def full_8d_pi0_model(x, pi0_pts, fit_conv=False):

    pi0_pts[:, 2] = 0.
    
    if fit_conv:
        eg1, eg2, theta1, theta2, c1, c2, f1, f2 = x
    else:
        c1, c2 = 86,22
        eg1, eg2, theta1, theta2, f1, f2 = x    

    return pi0_model.pi0_probability_binned(shower_pts=pi0_pts, eg1=eg1, eg2=eg2, theta1=theta1, theta2=theta2, conv_dist1=c1, conv_dist2=c2, f1=f1, f2=f2)

def full_6d_pi0_model(x, pi0_pts, c1,c2, fit_conv, use_ana):
    
    pi0_pts[:, 2] = 0.    
    
    if fit_conv:
        eg1, eg2, theta2, c1, c2 = x
        theta1 = np.radians(10.11)
    else:
        eg1, eg2, theta2 = x
        #eg1, eg2 = x
        theta1 = np.radians(10.11) #np.radians(1.11)
        #open_angle = np.arccos(1. - 135.**2 / (2. * eg1 * eg2))
        #theta2 = theta1 + open_angle

    f1,f2 = 150.,150.
    
    if use_ana:
        return pi0_model.pi0_probability(shower_pts=pi0_pts, eg1=eg1, eg2=eg2, phi=0, theta1=theta1, theta2=theta2, conv_dist1=c1, conv_dist2=c2)
    else:
        return pi0_model.pi0_probability_binned(shower_pts=pi0_pts, eg1=eg1, eg2=eg2, theta1=theta1, theta2=theta2, conv_dist1=c1, conv_dist2=c2, f1=f1, f2=f2)

def full_5d_pi0_model(x, pi0_pts, fit_conv=False):    

    pi0_pts[:, 2] = 0.
    
    if fit_conv:
        epi0, cos_pi0, a, c1, c2 = x
    else: 
        c1, c2 = 86, 22 # 220
        epi0, cos_pi0, a = x

    alpha, theta1, theta2 = futil.pi0_angles(epi0=epi0, cos_pi0=cos_pi0, a=a)
    eg1 = epi0 * a
    eg2 = epi0 - eg1
    f1,f2 = 150.,150.
        
    return pi0_model.pi0_probability_binned(shower_pts=pi0_pts, eg1=eg1, eg2=eg2, theta1=theta1, theta2=theta2, conv_dist1=c1, conv_dist2=c2, f1=f1, f2=f2)

def mcmc_full_5d_pi0_model(pi0_pts, epi0, cos_pi0, a):    

    pi0_pts[:, 2] = 0.
    c1, c2 = 11, 14  # 298

    alpha, theta1, theta2 = futil.pi0_angles(epi0=epi0, cos_pi0=cos_pi0, a=a)
    eg1 = epi0 * a
    eg2 = epi0 - eg1
    f1,f2 = 150.,150.
        
    return -pi0_model.pi0_probability_binned(shower_pts=pi0_pts, eg1=eg1, eg2=eg2, theta1=theta1, theta2=theta2, conv_dist1=c1, conv_dist2=c2, f1=f1, f2=f2)

def mcmc_3d_pi0_model(x, pi0_pts):
    pi0_pts[:, 2] = 0.
    
    theta1 = np.radians(10.31)
    Theta2 = [np.radians(18.8)] # 298
    
    # eg1, eg2, theta2 = x
    eg1, eg2 = x

    f1,f2 = 150.,150.

    if eg1 < 800 or eg1 > 1200 or eg2 < 800 or eg2 > 1200 or theta2 < np.radians(14) or theta2 > np.radians(25):
        return -np.inf

    # function returns -LL but need LL
    return pi0_model.pi0_probability_binned(shower_pts=pi0_pts, eg1=eg1, eg2=eg2, theta1=theta1, theta2=theta2, conv_dist1=c1, conv_dist2=c2, f1=f1, f2=f2)

def im_wrapper_2d_pi0_model(pi0_pts, eg1, eg2, theta1, theta2, c1, c2):
    pi0_pts[:, 2] = 0.

    f1,f2 = 150.,150.

    #if eg1 < 800 or eg1 > 1200 or eg2 < 800 or eg2 > 1200 or theta2 < np.radians(14) or theta2 > np.radians(25):
    #    return -np.inf

    # function returns -LL but need LL 
    # e^{-(-ll)} = e^{ll} = l
    
    res = pi0_model.pi0_probability_binned(shower_pts=pi0_pts, eg1=eg1, eg2=eg2, theta1=theta1, theta2=theta2, conv_dist1=c1, conv_dist2=c2, f1=f1, f2=f2)
    #res = pi0_model.pi0_probability(shower_pts=pi0_pts, eg1=eg1, eg2=eg2, phi=0, theta1=theta1, theta2=theta2, conv_dist1=c1, conv_dist2=c2)
    #print("E1/E2/F", eg1, "/", eg2, "/", res)
    return np.array([np.exp(-res)])


def print_fit_results(all_event_record, minr, event, fit_pi0_var=True, fit_conv=True):    
    print("******************************************")
    print("******************************************")
    print("True Gamma Eγ:", all_event_record["pi0_gamma_e_initial", event])
    print("True OA:", np.degrees(all_event_record["pi0_gamma_open_angle_initial", event]))
    print("++++++++++++++++++++++++++++++")
    print("All Param:", minr.x)
    
    if fit_pi0_var:
        print("Epi0", minr.x[0])
        print("cosθ_pi0", minr.x[1]) 
        print("a", minr.x[2])
        print("α/θ1/θ2", np.degrees(futil.pi0_angles(epi0=minr.x[0], cos_pi0=minr.x[1],  a=minr.x[2])))
        print("E1/E2", minr.x[0]*minr.x[2], "/", minr.x[0] - minr.x[0]*minr.x[2])
        if fit_conv:
            print("C1", minr.x[3])
            print("C2", minr.x[4])
        print("Min", minr.fun)
    else:
        print("E1", minr.x[0])
        print("E2", minr.x[1])
        print("θ1", np.degrees(minr.x[2]))
        #print("θ2", np.degrees(minr.x[3]))
        #print("OA", np.degrees(abs(minr.x[2]-minr.x[3])))
        print("OA", np.degrees(abs(minr.x[2]-np.radians(10.3))))
        if fit_conv:
            print("C1", minr.x[3])
            print("C2", minr.x[4])
        print("Min", minr.fun)

def get_pi0_event(all_event_record, event):
    use_truth_rotation = False
    pi0_pts_tmp, _, _, _ = pi0_transform.reco_transform_to_pi0_plane(event_record=all_event_record[event], evt=event)
    
    if use_truth_rotation:
        pi0_pts = truth_pts.T
        charge = ak.to_numpy(all_event_record["reco_all_spacePts_Integral", event])
        charge = charge.reshape(len(charge), 1)
        pi0_pts = np.hstack((pi0_pts, charge))
    else:
        pi0_pts = pi0_pts_tmp

    return pi0_pts

# Define the function to fit (a Gaussian)
def gaussian(x, cen, wid, amp):
    return amp * np.exp(-(x-cen)**2 / (2*wid**2))

def norm_gaussian(x, cen, wid, amp):
    wid = abs(wid)
    return (1. / (wid*np.sqrt(2.*np.pi)))  * np.exp(-(x-cen)**2 / (2*wid**2))

def fit_gaussian_width(hist, bin_edges, label, color, height_frac, text_y, down=-1, up=1):
    half_height_mask = hist >= np.max(hist) * height_frac
    #print(half_height_mask)
  
    # Fit the histogram data with the Gaussian function
    bin_centers = (bin_edges[1:] + bin_edges[:-1]) / 2
    popt, pcov = curve_fit(gaussian, bin_centers[half_height_mask], hist[half_height_mask]/np.max(hist[half_height_mask]))
    print("μ", popt[0], "σ", popt[1])

    x = np.linspace(down,up, 200)
    y = norm_gaussian(x, *popt)
    plt.plot(x, y, linestyle='--', label=label, color=color)
    plt.text(-1, text_y, label + ":  $\mu$ = " + str(round(popt[0], 4)) + "  $\sigma$ = " + str(round(popt[1], 4)), fontsize=16)

def shower_func(x, A, k, mu, sigma, reverse=False, normalize=False):
    """
    Taken from https://arxiv.org/pdf/1603.08591.pdf
    """
    exp_arg = (x - mu) / sigma
    
    if reverse:
        if normalize:
            a1 = sigma * np.sqrt(np.pi/2.) * (math.erf(k/np.sqrt(2)) + math.erf(mu/(sigma*np.sqrt(2))))
            a2 = (sigma / k) * np.exp(-0.5 * k**2)
            norm = a1 + a2
        else:
            norm = 1.
        return A * np.where(x <= mu + k*sigma, np.exp(-0.5 * exp_arg ** 2), np.exp(0.5 * (k ** 2) - k * (exp_arg))) / norm
    else:  # original
        if normalize:
            a1 = -(sigma / k) * np.exp(-0.5 * k**2) * (np.exp(k * (k - (mu / sigma))) - 1.)
            a2 = sigma * np.sqrt(np.pi/2.) * (math.erf(k/np.sqrt(2)) + 1.)
            norm = a1 + a2
        else:
            norm = 1.
        return A * np.where(x >= mu - k*sigma, np.exp(-0.5 * exp_arg ** 2), np.exp(0.5 * (k ** 2) + k * (exp_arg))) / norm

def annealer_fit_results(minimizer, evt_egamma, fit_conv):
    if fit_conv:
        e1,e2,t1,t2,c1,c2 = minimizer.x[0], minimizer.x[1], np.radians(10.11), minimizer.x[2], minimizer.x[3], minimizer.x[4]
    else:
        e1,e2,t1,t2 = minimizer.x[0], minimizer.x[1], np.radians(10.11), minimizer.x[2]
    #e1,e2,t1 = minimizer.x[0], minimizer.x[1], np.radians(10.11)
    #t2 = t1 + np.arccos(1. - 135.**2 / (2. * e1 * e2))
    print("Min", minimizer.fun)
    print("E1/E2", round(e1,2), "/", round(e2,2))
    print("θ1/θ2/α", round(np.degrees(t1),2), "/", round(np.degrees(t2),2), round(np.degrees(abs(t1-t2)),2))
    print("E Res", round((e1+e2)/np.sum(evt_egamma) - 1., 4) * 100, "[%]") 
    if fit_conv: print("C1/C2", round(c1,2), "/", round(c2,2))

    if fit_conv:
        return (e1,e2,t1,t2,c1,c2)
    else:
        return (e1,e2,t1,t2)
        
def calo_energy(qtotal, epi0):
    e1,e2 = epi0/2., epi0/2.
    k1, mu1, sigma1 = calo_params(e0=e1, pdict=qtotal_fit_params)
    k2, mu2, sigma2 = calo_params(e0=e2, pdict=qtotal_fit_params)
    
    sigma_tot = np.sqrt((sigma1*mu1)**2 + (sigma2*mu2)**2)
    q_prob = shower_func(x=qtotal, A=1., k=k1+k2, mu=mu1+mu2, sigma=sigma_tot, reverse=False, normalize=True)
    
    return q_prob 
    
def fit_pi0_calo(pi0_pts):
    elist = np.linspace(1600,2400, 80)
    qtotal = np.sum(pi0_pts[:, 3][pi0_pts[:, 3] > -1.])
    nll_list = [-np.log(calo_energy(qtotal=qtotal, epi0=e)) for e in elist]
 
    return elist[np.argmin(nll_list)]

def calo_params(e0, pdict):
    k = pdict["k"][0] * e0 + pdict["k"][1]
    mu = pdict["mu"][0] * e0 + pdict["mu"][1]
    sigma = pdict["sigma"][0] * e0 + pdict["sigma"][1]
    return k, mu, sigma

pdf_hist_file = "/Users/jsen/work/Protodune/analysis/pi0_fitter/etc/qtotal_fit_params_v1.pkl"
with open(pdf_hist_file, 'rb') as ifile:
    qtotal_fit_params = pickle.load(ifile)

pdf_hist_file = "/Users/jsen/work/Protodune/analysis/pi0_fitter/etc/q3_fit_params_v1.pkl"
with open(pdf_hist_file, 'rb') as ifile:
    q3_fit_params = pickle.load(ifile)
    

In [10]:
def fit_pi0_event(pi0_pts, c1, c2, use_ana, fit_conv):
    fit_pi0_var = False
    #fit_conv = False
    float_energy = False
    
    temp = 10000#15000
    maxiter = 1000
    
    epi0_bound = (1600,2400)
    a_bound = (0.34,0.8)
    cos_bound = (0.55,1.)
    eg1_bound = (800,1200)
    eg2_bound = (800,1200)
    theta1_bound = (np.radians(5),np.radians(13))
    theta2_bound = (np.radians(14),np.radians(30)) #(np.radians(-25),np.radians(-5))
    c1_bound = (0,80)
    c2_bound = (0,80)
    f1_bound = (120,2500)
    f2_bound = (120,2500)
    
    if fit_pi0_var:
        if fit_conv:
            bounds = [epi0_bound, cos_bound, a_bound, c1_bound, c2_bound]
            start_pt = np.array([1800,0.95, 0.55, 55, 10])
        else:
            bounds = [epi0_bound, cos_bound, a_bound]
            start_pt = np.array([1800,0.98, 0.95])
    else:
        if fit_conv:
            bounds = [eg1_bound, eg2_bound, theta2_bound, c1_bound, c2_bound]
            start_pt = np.array([900,1000,np.radians(19.), 3, 8])
        else:        
            if float_energy:
                bounds = [eg1_bound, eg2_bound, theta1_bound, theta2_bound, f1_bound, f2_bound]
                start_pt = np.array([900,1000, np.radians(10),np.radians(25),800, 1000])
            else:
                #bounds = [eg1_bound, eg2_bound, theta1_bound, theta2_bound]
                #start_pt = np.array([900,1000, np.radians(10),np.radians(15)])
                bounds = [eg1_bound, eg2_bound, theta2_bound]
                start_pt = np.array([900,1000, np.radians(19.)])

    print("Start Point:", start_pt)
    
    if fit_pi0_var:
        min_result = dual_annealing(full_5d_pi0_model, args=np.array([pi0_pts]), bounds=bounds, maxiter=maxiter, x0=start_pt, initial_temp=temp)
    else:
        if float_energy:    
            min_result = dual_annealing(full_8d_pi0_model, args=np.array([pi0_pts]), bounds=bounds, maxiter=maxiter, x0=start_pt, initial_temp=temp)
        else:            
            min_result = dual_annealing(full_6d_pi0_model, args=np.array([pi0_pts,c1,c2,fit_conv,use_ana]), bounds=bounds, maxiter=maxiter, x0=start_pt, initial_temp=temp)
    
    return min_result
    

In [11]:
event = 298
theta1 = np.radians(10.11)
theta2 = np.radians(20.)#np.radians(18.8)

fit_results_calo = []
fit_results_ana = []
fit_results_bin = []
true_param = []

cutoff = 1

i = 1
for event in acceptable_event_dict.keys():
    print("i", i)
    if i > cutoff:
        break
        
    pi0_pts, _, _, truth_flipped = pi0_transform.reco_transform_to_pi0_plane(event_record=all_event_record[event], evt=event)
    #pi0_pts, _, _, truth_flipped = pi0_transform.truth_transform_to_pi0_plane(event_record=all_event_record[event], evt=event)
    pi0_pts[:, 2] = 0.

    print("++++++++++++++++++++++++++++++++++++")
    print("Event", event)
    print("True Gamma Eγ:", all_event_record["pi0_gamma_starte_initial", event])
    print("True OA:", np.degrees(all_event_record["pi0_gamma_open_angle_initial", event]))
    
    if truth_flipped:
        print("Flipped")
        true_c1, true_c2 = all_event_record["true_decay_gamma_conv_dist", event][1]+4,all_event_record["true_decay_gamma_conv_dist", event][0]+4
        true_e1, true_e2 = all_event_record["pi0_gamma_starte_initial", event][1],all_event_record["pi0_gamma_starte_initial", event][0]
    else:
        true_c1, true_c2 = all_event_record["true_decay_gamma_conv_dist", event][0]+4,all_event_record["true_decay_gamma_conv_dist", event][1]+4
        true_e1, true_e2 = all_event_record["pi0_gamma_starte_initial", event][0],all_event_record["pi0_gamma_starte_initial", event][1]

    print("True C1/C2", round(true_c1,2), "/", round(true_c2,2))


    print("Fitting Calo Model")
    calo_fit_energy = fit_pi0_calo(pi0_pts=pi0_pts)            
    print("Fitting Analytic Model")
    min_result_ana = fit_pi0_event(pi0_pts=pi0_pts, c1=true_c1, c2=true_c2, use_ana=True, fit_conv=False)
    print("Fitting Corr. Model")
    min_result_bin = fit_pi0_event(pi0_pts=pi0_pts, c1=true_c1, c2=true_c2, use_ana=False, fit_conv=True)
    

    print("---> Calo")
    print("E\pi", calo_fit_energy)
    print("E Res", 100*((calo_fit_energy / (true_e1 + true_e2))-1), "[%]")
    fit_results_calo.append(calo_fit_energy)
    print("---> Ana")
    fit_results_ana.append(annealer_fit_results(minimizer=min_result_ana, evt_egamma=all_event_record["pi0_gamma_starte_initial", event], fit_conv=False))
    print("---> Corr")
    fit_results_bin.append(annealer_fit_results(minimizer=min_result_bin, evt_egamma=all_event_record["pi0_gamma_starte_initial", event], fit_conv=True))
    
    true_param.append((true_e1, true_e2, np.degrees(theta1), np.degrees(theta1+all_event_record["pi0_gamma_open_angle_initial", event]), true_c1, true_c2))

    i += 1

fit_results_calo = np.asarray(fit_results_calo)
fit_results_ana = np.asarray(fit_results_ana)
fit_results_bin = np.asarray(fit_results_bin)
true_param = np.asarray(true_param)


i 1
++++++++++++++++++++++++++++++++++++
Event 34
True Gamma Eγ: [850, 898]
True OA: 8.863669885206003
Flipped
True C1/C2 31.97 / 22.11
Fitting Calo Model
Fitting Analytic Model
Start Point: [9.00000000e+02 1.00000000e+03 3.31612558e-01]


/var/folders/99/djgy5zlx1zs5t65y9tg4vkkc0000gn/T/ipykernel_23840/3135261622.py:50: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  min_result = dual_annealing(full_6d_pi0_model, args=np.array([pi0_pts,c1,c2,fit_conv,use_ana]), bounds=bounds, maxiter=maxiter, x0=start_pt, initial_temp=temp)
/Users/jsen/work/Protodune/analysis/pi0_fitter/pi0fit/pi0_model.py:270: RuntimeWarning: invalid value encountered in sqrt
  1 / np.sqrt(gamma ** 2 * np.sin(alpha / 2.) ** 2 - 1))


Fitting Corr. Model
Start Point: [9.00000000e+02 1.00000000e+03 3.31612558e-01 3.00000000e+00
 8.00000000e+00]


/Users/jsen/work/Protodune/analysis/pi0_fitter/pi0fit/shower_model.py:59: RuntimeWarning: divide by zero encountered in divide
  return (tmp_long * tmp_radial * (1. / np.sqrt(r))) / self.topo_normalization(energy)


---> Calo
E\pi 1600.0
E Res -8.435510030403792 [%]
---> Ana
Min 9.083436913359758
E1/E2 831.07 / 812.62
θ1/θ2/α 10.11 / 19.63 9.52
E Res -5.94 [%]
---> Corr
Min 88.14360550636012
E1/E2 820.63 / 801.38
θ1/θ2/α 10.11 / 19.76 9.65
E Res -7.180000000000001 [%]
C1/C2 32.83 / 15.81
i 2


In [ ]:
fit_resolution = True
density = True
hfrac = 0.01

bins = 40

shift = (2 / bins) / 2.
print("shift", shift)

###########
# E_pi0

plt.figure(figsize=(12,7))
h,b,_ = plt.hist((np.sum(fit_results_bin[:,0:2], axis=1)/np.sum(true_param[:,0:2], axis=1)) -1, bins=bins, range=[-1+shift, 1+shift], color='royalblue', density=density, label="Corr.")
if fit_resolution: fit_gaussian_width(hist=h, bin_edges=b, label="Corr. Fit", color="blue", height_frac=hfrac,text_y=6)
#h,b,_ = plt.hist((np.sum(fit_results_ana[:,0:2], axis=1)/np.sum(true_param[:,0:2], axis=1)) -1, bins=bins, range=[-1+shift, 1+shift], alpha=1, color='indianred', density=density, label="Analytic")
#if fit_resolution: fit_gaussian_width(hist=h, bin_edges=b, label="Ana Fit", color="red", height_frac=hfrac,text_y=5)
h,b,_ = plt.hist((fit_results_calo/np.sum(true_param[:,0:2], axis=1)) -1, bins=bins, range=[-1+shift, 1+shift], alpha=0.8, color='forestgreen', density=density, label="Calo")
if fit_resolution: fit_gaussian_width(hist=h, bin_edges=b, label="Calo Fit", color="green", height_frac=hfrac,text_y=4)
plt.xlabel("$E_{\pi^0}$:  Reco/True - 1", fontsize=16)
plt.legend()
plt.show()

###########
# E_1

hfrac = 0.12
print("True", true_param[:,0])
print("Reco", fit_results_bin[:,0])
print("Reco/True -1", (fit_results_bin[:,0]/true_param[:,0]) -1)
plt.figure(figsize=(12,7))
#h,b,_ = plt.hist((np.max(fit_results_bin[:,0:2],axis=1)/np.max(true_param[:,0:2],axis=1)) -1, bins=bins, range=[-1+shift, 1+shift], color='royalblue', density=density, label="Corr.")
h,b,_ = plt.hist((fit_results_bin[:,0]/true_param[:,0]) -1, bins=bins, range=[-1+shift, 1+shift], color='royalblue', density=density, label="Corr.")
if fit_resolution: fit_gaussian_width(hist=h, bin_edges=b, label="Corr. Fit", color="blue", height_frac=hfrac,text_y=3)
h,b,_ = plt.hist((fit_results_ana[:,0]/true_param[:,0]) -1, bins=bins, range=[-1+shift, 1+shift], alpha=0.9, color='indianred', density=density, label="Analytic")
if fit_resolution: fit_gaussian_width(hist=h, bin_edges=b, label="Ana Fit", color="red", height_frac=hfrac,text_y=2)
plt.xlabel("$E1_{\gamma}$:  Reco/True - 1", fontsize=16)
plt.legend()
plt.show()

###########
# E_2

hfrac = 0.1
print("True", true_param[:,1])
print("Reco", fit_results_bin[:,1])
print("Reco/True -1", (fit_results_bin[:,1]/true_param[:,0]) -1)
plt.figure(figsize=(12,7))
h,b,_ = plt.hist((fit_results_bin[:,1]/true_param[:,1]) -1, bins=bins, range=[-1+shift, 1+shift], color='royalblue', density=density, label="Corr.")
if fit_resolution: fit_gaussian_width(hist=h, bin_edges=b, label="Corr. Fit", color="blue", height_frac=hfrac,text_y=4)
h,b,_ = plt.hist((fit_results_ana[:,1]/true_param[:,1]) -1, bins=bins, range=[-1+shift, 1+shift], alpha=0.9, color='indianred', density=density, label="Analytic")
if fit_resolution: fit_gaussian_width(hist=h, bin_edges=b, label="Ana Fit", color="red", height_frac=hfrac,text_y=3)
plt.xlabel("$E2_{\gamma}$:  Reco/True - 1", fontsize=16)
plt.legend()
plt.show()

plt.figure(figsize=(10,5))
plt.hist(np.degrees(fit_results_bin[:,3]) - true_param[:,3], bins=40, range=[-4.875,5.125], color='royalblue', label="Corr.")
plt.hist(np.degrees(fit_results_ana[:,3]) - true_param[:,3], bins=40, range=[-4.875,5.125], alpha=0.8, color='indianred', label="Analytic")
plt.xlabel("Reco - True [deg]")
plt.legend()
plt.show()


plt.hist2d((fit_results_bin[:,0]/true_param[:,0]) -1, (fit_results_bin[:,1]/true_param[:,1]) -1, bins=[bins, bins], range=[[-1+shift, 1+shift],[-1+shift, 1+shift]], cmap=plt.cm.jet, cmin=1)
plt.xlabel("$E1_{\gamma}$:  Reco/True - 1", fontsize=16)
plt.ylabel("$E2_{\gamma}$:  Reco/True - 1", fontsize=16)
# plt.xlim(-0.25, 0.25)
# plt.ylim(-0.25, 0.25)
plt.colorbar()
plt.show()